# Identifying Empty Citibike Docks

Ultimately, the goal behind this Citibike project work is to create 
insightful analyses around the Citibike system. I'd also like to use the 
results of the analyses to build some novel predictive models. There are 
plenty of "gotchas" with these data and potential analyses, and I'm going
to address the major ones in their own notebooks. 

In this particular notebook, we address the issue of discerning what a 
lack of rides originating from a given station really means. 

The data only contains information about individual rides, not the 
availability of bikes at a station. Zero rides originating from a station 
in a certain time period can mean one of two things:
* There is no demand for bikes from the station. 
* There is such high demand for bikes that station is empty.  

The latter situation poses a particular issue for any kind of demand 
model, as the actual ridership data does not reflect the number of people
who would have started a ride had a bike been available to them.

Furthermore, knowing which stations frequently run out of bikes is useful 
in terms of knowing where to expand stations, build new ones, or when and
how to stage resupply runs from other areas. 

## Setup
Module imports and notebook setup. 

In [40]:
from datetime import timedelta
import os
import pandas as pd

## Getting the Data
If you want to run all this yourself and haven't run through the Citibike
Data Pull notebook, you should do so now.

Our data is stored in /data as monthly .parquet files containing "fact 
tables" of ridership data. /data also contains the relevant "dimension
tables" that can join to the ridership data. It was done this way to 
significantly reduce the size of the data such that we can access larger
time frames and do more useful research. 

Let's start by reading our dimension tables into the notebook. 

### Stations
These are the docks for the bikes. Each station has an ID, a cross street, 
and a lat/long.

In [15]:
stations = pd.read_parquet(os.path.join('../data/stations.parquet'))
stations.head(3)

,int_station_id,station_id,station_name,lat,lng
0,0,6283.05,48 St & Skillman Ave,40.746155,-73.916191
1,1,5105.01,Liberty St & Broadway,40.708858,-74.010231
2,2,6809.07,W 56 St & 6 Ave,40.763405,-73.977226


### Rideable Types
These are the different kinds of bikes. Currently that's limited to 
electric (pedal assisted) and classic (regular) bikes. 

In [16]:
rideables = pd.read_parquet(os.path.join('../data/rideable_types.parquet'))
rideables.head(3)

,rideable_id,rideable_type
0,0,electric_bike
1,1,classic_bike


### Membership Types
These are the kinds of rider "memberships" available. Basically this 
denotes whether a rider has a monthly subscription to the Citibike 
service or if they're riding on a "casual" pay-as-you-go basis. 

In [17]:
membership_types = pd.read_parquet(os.path.join('../data/membership_types.parquet'))
membership_types.head(3)

,membership_id,membership_type
0,0,member
1,1,casual


### Ride Data
Now let's look at a sample of our fact table formatted ride data. 

In [27]:
# All the ride files start with a yyyy or yyyymm before an underscore
# We can extract those filenames from the data folder
ride_filepaths = [f for f in os.listdir('../data') if f.split('_')[0].isnumeric()]

# Concat ride files into a single dataframe
rides = pd.DataFrame()
for path in ride_filepaths:
    rides = pd.concat([rides, pd.read_parquet(f'../data/{path}')])

rides.head(3)

,ride_id,started_at,start_station_id,end_station_id,rideable_id,membership_id,trip_duration
0,0FC89A53DF9D7E90,2024-03-07 19:49:43,0,70,0,0,1850
1,0FF38F5D1277746B,2024-03-15 17:45:30,1,68,0,0,609
2,DE040AD144FB0BFA,2024-03-19 18:00:52,2,65,0,0,394


## Identifying Empty Docks

This might seem reasonably straightforward at first glance. We could 
loop through undocking/docking events 

In [58]:
def get_dock_capacity(rides:pd.DataFrame, station_id:int) -> int:
    """
    Calculates the maximum capacity of a Citibike dock.

    Args:
        rides DataFrame: Individual rides data. 
        station_id int: Station to figure out the max capacity for. 
    """
    # Add docking time to the rides data
    station_events = rides.loc[(rides['start_station_id'] == station_id) | (rides['end_station_id'] == station_id)]
    station_events.loc[:, 'started_at'] = pd.to_datetime(station_events['started_at'])
    station_events.loc[:, 'ended_at'] = [dt + timedelta(seconds=dur) for dt, dur in zip(station_events['started_at'], station_events['trip_duration'])]

    # Get undock events 
    undock_events = station_events.loc[station_events['start_station_id'] == station_id][['started_at']]
    undock_events = undock_events.rename(columns={'started_at': 'dt'})
    undock_events.loc[:, 'action'] = 'undock'
    # Get dock events
    dock_events = station_events.loc[station_events['end_station_id'] == station_id][['ended_at']]
    dock_events = dock_events.rename(columns={'ended_at': 'dt'})
    dock_events.loc[:, 'action'] = 'dock'

    station_events = pd.concat([undock_events, dock_events])
    station_events = station_events.sort_values('dt')

    min_docked_bikes = 0
    max_docked_bikes = 0
    current_docked_bikes = 0

    for dt, action in station_events.to_records(index=False):
        if action == 'dock':
            current_docked_bikes += 1
        elif action == 'undock':
            current_docked_bikes -= 1

        min_docked_bikes = min(min_docked_bikes, current_docked_bikes)
        max_docked_bikes = max(max_docked_bikes, current_docked_bikes)


    return current_docked_bikes


output = get_dock_capacity(rides, 0)
output


/var/folders/yb/nf2crlgx1kdgc_cpn2rkwzfm0000gn/T/ipykernel_27271/2251594316.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_events.loc[:, 'ended_at'] = [dt + timedelta(seconds=dur) for dt, dur in zip(station_events['started_at'], station_events['trip_duration'])]


2741